In [2]:
import pandas as pd

# Load the dataset
df = pd.read_csv("phishing_site_urls.csv")  # Replace with your correct filename

# Check the actual column names
print(df.columns)

# Option 1: Rename all three columns properly
df.columns = ["Index", "EmailText", "Label"]

# Option 2: Skip the unnamed first column if it's just the index
df = pd.read_csv("phishing_site_urls.csv", index_col=0)
df.columns = ["EmailText", "Label"]

# Encode labels: 'Phishing Email' → 1, 'Safe Email' → 0
df["Label"] = df["Label"].map({"Phishing Email": 1, "Safe Email": 0})

# Display a preview
print(df.head())


Index(['Unnamed: 0', 'Email Text', 'Email Type'], dtype='object')
                                           EmailText  Label
0  re : 6 . 1100 , disc : uniformitarianism , re ...      0
1  the other side of * galicismos * * galicismo *...      0
2  re : equistar deal tickets are you still avail...      0
3  \nHello I am your hot lil horny toy.\n    I am...      1
4  software at incredibly low prices ( 86 % lower...      1


In [4]:
df['EmailText'] = df['EmailText'].fillna('')


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X = vectorizer.fit_transform(df['EmailText'])
y = df['Label']


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train, y_train)


MultinomialNB()

In [8]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.94      0.97      0.96      2273
           1       0.96      0.91      0.93      1457

    accuracy                           0.95      3730
   macro avg       0.95      0.94      0.94      3730
weighted avg       0.95      0.95      0.95      3730



In [9]:
import joblib
joblib.dump(model, 'phishing_email_model.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')


['tfidf_vectorizer.pkl']

In [10]:
import joblib

# Load the saved model and vectorizer
model = joblib.load('phishing_email_model.pkl')
vectorizer = joblib.load('tfidf_vectorizer.pkl')


In [11]:
# Example test email
test_email = ["Congratulations! You’ve won a free iPhone. Click here to claim now."]

# Transform the text using the vectorizer
X_test_email = vectorizer.transform(test_email)

# Predict
prediction = model.predict(X_test_email)

# Output
if prediction[0] == 1:
    print("Prediction: 🚨 Spam/Phishing Email")
else:
    print("Prediction: ✅ Not Spam")


Prediction: 🚨 Spam/Phishing Email


In [12]:
test_emails = [
    "Hello, please find the attached report for last quarter.",
    "You have been selected for a prize! Send your bank details.",
    "Meeting rescheduled to 3 PM today."
]

X_test = vectorizer.transform(test_emails)
predictions = model.predict(X_test)

for email, label in zip(test_emails, predictions):
    print(f"\nEmail: {email}")
    print("Prediction:", "🚨 Spam" if label == 1 else "✅ Not Spam")



Email: Hello, please find the attached report for last quarter.
Prediction: ✅ Not Spam

Email: You have been selected for a prize! Send your bank details.
Prediction: 🚨 Spam

Email: Meeting rescheduled to 3 PM today.
Prediction: ✅ Not Spam
